# PROJECT GEGEVENSBANKEN - SQL

# Imports

In [1]:
# Setup
import getpass  # Package om een paswoordveldje te genereren.
import json  # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import mysql.connector  # MySQL package
import numpy as np
import os
import pandas as pd  # Populaire package voor data-verwerking
import re
import sys

from inspect import getsource
from IPython.display import display, Markdown
from os.path import dirname

# Imports van binnen onze eigen repository
this_dir = os.getcwd()
root_dir = dirname(this_dir)
docs_dir = os.path.join(root_dir, "docs")
source_dir = os.path.join(root_dir, "src")
script_dir = os.path.join(root_dir, "scripts")
solution_dir = os.path.join(root_dir, "solution")

sys.path.append(script_dir)
sys.path.append(source_dir)

import db_project  # onze eigen codebase in deze repository
from db_project import (
    verbind_met_GB,
    run_query,
    res_to_df,
)  # 3 basic functionalities, explained in introduction.

# Text files from which some descriptions are rendered (this helps us to keep everything perfectly consistent across different notebooks etc)
introduction_file = os.path.join(docs_dir, "01-introduction.md")
instructions_file = os.path.join(docs_dir, "02-instructions.md")
qry_description_file = os.path.join(docs_dir, "03-qry-descriptions.md")
submissions_file = os.path.join(docs_dir, "04-submissions.md")

from db_project import parse_markdown

In [2]:
display(
    Markdown(parse_markdown(introduction_file, section_number=[1, 2, 3]))
)  # Tekst en uitleg worden ingelezen uit bestanden uit de `docs` folder.

# 1. Inleiding

Cf. het document `prerequisites.pdf` in de `docs` folder. 

## 1.1 Python packages

De import statements (cf. boven) vormen de standaardconfiguratie van python-packages die we gebruiken. _Van deze configuratie afwijken is dan ook op eigen risico, gezien dat betekent dat je ook afwijkt van de configuratie van de computer waarop wij jullie oplossingen runnen._

## 1.2 Interageren met een gegevensbank

Naast de standaard packages, importeren we ook drie zelfgemaakte functies. Deze implementeren functionaliteiten die je vaak zal nodig hebben als je vanuit python queries naar je gegevensbank wil sturen: `verbind_met_GB`, `run_query` en `res_to_df`. Hun source code (en documentatie) bekijken geeft de nodige informatie over hoe ze te gebruiken:
    
    


In [3]:
print(getsource(verbind_met_GB))
print("- --- - --- - --- - --- - --- - --- - - --- - --- - --- - --- - --- - --- -\n")
print(getsource(run_query))
print("- --- - --- - --- - --- - --- - --- - - --- - --- - --- - --- - --- - --- -\n")
print(getsource(res_to_df))

def verbind_met_GB(username, hostname, gegevensbanknaam, password=None):
    """
    Maak verbinding met een externe gegevensbank

    Parameters
    ----------

    username:           str
                        Username van de gebruiker
    hostname            str
                        Naam van de host. In het geval van lokale server 'localhost'
    gegevensbanknaam    str
                        Naam van de gegevensbank
    password            str, None
                        Wachtwoord kan al meegegeven worden. Indien niet, wordt
                        een wachtwoordveldje gegenereerd waar de gebruiker het
                        kan ingeven.
    Returns
    -------
    connection          connection object
                        Dit is het soort object dat wordt teruggeven door
                        connect() methods van packages die voldoen aan de DB-API

    """

    if password is None:
        password = getpass.getpass()  # Genereer vakje voor wachtwoord in te geven
 

In [4]:
display(Markdown(parse_markdown(introduction_file, section_number=4)))

## 1.3 Kolomnamen en input parameters

We leggen op voorhand reeds de **kolomnamen van de oplossingen**, en de **naam en types van de inputparameters** vast. Hier moet je je dus aan houden en mag je dus niks aan wijzigen.

- Het aantal kolommen (en hun volgorde) van jullie oplossing en de onze moeten exact overeen komen, vandaar dat de kolomnamen hieronder gegeven zijn. Dit komt trouwens van pas bij het opstellen van je queries! 
    
- Dankzij de voorbeeldparameters, die al gegeven zijn in de functiedefinitie e.g.: `query_42(connection, col_names, super_voorbeeldparam = 101)` weten jullie exact welke vorm en type (integer, lijst, etc) de inputparameters moeten hebben. Wijzig zeker niets aan de naam van die parameters (*super_voorbeeldparam* blijft *super_voorbeeldparam*). De default waarden geven op hun beurt dan weer informatie over het type van de inputparameters. Let wel: de (soms onnozele) default waarden zijn _louter ter illustratie_, zorg ervoor dat je query ook met andere waarden dan de defaults werkt!

**Samengevat: oplossingen die deze vorm niet respecteren, zullen crashen op onze machines en resulteren in een score van 0 op die query.**


In [5]:
# Inspecteer de dictionairy van kolomnamen

filename = os.path.join(solution_dir, "all_q_colnam.json")
col_names = json.load(open(filename, "r"))
 
# Inspecteer dictionary
for k,v in col_names.items():
    print("""De kolomnamen van {}: {}
    """.format(k,v))

De kolomnamen van query_01: ['tname', 'year', 'HomeRun']
    
De kolomnamen van query_02: ['nameFirst', 'nameLast', 'birthYear', 'birthMonth', 'birthDay']
    
De kolomnamen van query_03: ['nameFirst', 'nameLast', 'tname']
    
De kolomnamen van query_04: ['teamID', 'name', 'yearID', 'W', 'L', 'MaxSalary']
    
De kolomnamen van query_05: ['teamID', 'name', 'yearID', 'rank']
    
De kolomnamen van query_06: ['playerID', 'nameFirst', 'nameLast']
    
De kolomnamen van query_07: ['playerID', 'nameFirst', 'nameLast', 'teamID']
    
De kolomnamen van query_08: ['teamID', 'tname', 'bestyear', 'HR', 'W']
    
De kolomnamen van query_09: ['teamID', 'tname', 'yearID', 'rank', 'HR']
    
De kolomnamen van query_10: ['teamId', 'tname', 'HOFplayercount']
    


In [6]:
display(Markdown(parse_markdown(instructions_file, section_number=[1, 2])))

# 2. Instructies bij gebruik van de notebook

Hieronder volgen 10 onvolledige functies (e.g., `query_42(connection, col_names, super_voorbeeldparam = ['joske', 'jef'])`). Aan jullie om ze aan te vullen zodat de functie:  

1. Een corecte query opstelt
2. De query uitvoert op de database
3. Het resultaat teruggeeft in een DataFrame.

Voor stap 2 en 3 zijn de nodige functies al voorhanden, i.e.: `run_query(connection, query)` en `res_to_df(res, column_names)`. Jullie werk zal dus vooral bestaan uit stap 1, queries opstellen. Elke functie heeft minstens 2 inputargumenten:

1. `connection`:   Een connection object 
2. `column_names`: De kolomnamen van het Pandas DataFrame
    
Gevolgd door eventuele extra argumenten (e.g., `super_voorbeeldparam = ['joske','jef']`) die dienen om parameters in te query te injecteren. 

**Nogmaals: verander niets aan de namen van de functies, namen van de functie-argumenten en de kolomnamen van de resulterende DataFrames. Wijzigingen hieraan leiden onvermijdelijk tot een score van 0 op die query.**

Je kan naar believen extra cellen toevoegen om je queries te testen, resultaten te inspecteren etc. Daar dient deze notebook immers voor, deze notebook wordt **niet ingediend** als oplossing.

We vragen jullie om de finale, ingevulde functies te kopiëren naar een extern python script dat _enkel en alleen_ deze ingevulde functies bevat. Cf. de laatste sectie van deze notebook voor instructies omtrent hoe in te dienen.

## 2.1 Voorbeeld-query opstellen

Om jullie al wat op weg te zetten volgt hier een voorbeeldje over hoe je te werk kan gaan.

**Beschrijving**

Het resultaat van deze functie is een Pandas DataFrame met teamnaam, jaar en aantal homeruns van teams die meer dan een gegeven aantal `homeruns` hadden in dat jaar.

Sorteer aflopend op aantal homeruns.

**Oplossing**



In [7]:
# Voorbeeld-query (oplossing)
def query_EX(connection, column_names, homeruns=20):
    # Bouw je query
    query = """
    select    t.name, t.yearID, t.HR
    from      Teams as t
    where     t.HR > {}
    order by  t.HR DESC;
    """.format(
        homeruns
    )  # TIP: Zo krijg je parameters in de string (samen met `{}` in de string)

    # Stap 2 & 3
    res = run_query(connection, query)  # Query uitvoeren
    df = res_to_df(res, column_names)  # Query in DataFrame brengen

    return df

In [8]:
display(Markdown(parse_markdown(instructions_file, section_number=3)))

## 2.2 Voorbeeld-query runnen

Om een query te runnen maken we gebruik van de hulpfunctie die we eerder ter beschikking stelden (e.g. `verbind_met_GB`). Concreet bestaat dit proces uit twee stappen:

1. Eerst maken we een verbindingsobject met de databank
2. Vervolgens runnen we onze query, en inspecteren we het resultaat.


In [9]:
# STAP 01: Verbindingsobject aanmaken

username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'db-project'          # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db, password='')

In [10]:
# STAP 02: Query runnen, resultaat ophalen en inspecteren.

kolomnamen_voorbeeldquery = col_names['query_01']        # Voorbeeldquery heeft dezelfde kolomnamen als query 1.

df = query_EX(c,kolomnamen_voorbeeldquery,homeruns=10)   # Query-functie uitvoeren, geeft resultaat van de query in een DataFrame

df.tail()                                                # Inspecteer laatste resultaten (laat. tail() weg om alles te zien)

,tname,year,HomeRun
2689,Cleveland Naps,1907,11
2690,St. Louis Browns,1884,11
2691,Detroit Tigers,1907,11
2692,Chicago White Sox,1905,11
2693,Philadelphia Phillies,1908,11


In [11]:
# EXTRA: Query runnen met andere parameterwaarde

kolomnamen_voorbeeldquery = col_names['query_01']        # Voorbeeldquery heeft dezelfde kolomnamen als query 1.

df = query_EX(c,kolomnamen_voorbeeldquery,homeruns=100)  # Query-functie uitvoeren, geeft resultaat van de query in een DataFrame

df.tail()                                                # Inspecteer laatste resultaten (laat. tail() weg om alles te zien)

,tname,year,HomeRun
1519,Cleveland Indians,1940,101
1520,Detroit Tigers,1976,101
1521,Philadelphia Phillies,1970,101
1522,St. Louis Cardinals,1980,101
1523,Seattle Mariners,2010,101


## 3. Queries

In [12]:
display(Markdown(parse_markdown(qry_description_file, section_number=1)))

## 3.1 Query 01

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe met: de teamnaam, het jaar, en het aantal homeruns per team, en dit voor alle teams.

Sorteer aflopend op het aantal homeruns.



In [13]:
def query_01(connection, column_names):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [14]:
display(Markdown(parse_markdown(qry_description_file, section_number=2)))

## 3.2 Query 02

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe met: de voornaam, achternaam, geboortejaar, geboortemaand, geboortedag van spelers die hun eerste major league appearance maakten na een gegeven *`datum_x`*. 

Sorteer alfabetisch oplopend op achternaam.



In [15]:
def query_02(connection, column_names, datum_x='1980-01-16'):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [16]:
display(Markdown(parse_markdown(qry_description_file, section_number=3)))

## 3.3 Query 03

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat, per club: de clubnaam en de voor- en achternaam van alle managers weergeeft, die ooit voor de club gewerkt hebben als playermanager. Per club mag een welbepaalde manager slechts 1 keer in het resultaat voorkomen. 

Sorteer alfabetisch oplopend op clubnaam.




In [17]:
def query_03(connection, column_names):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [18]:
display(Markdown(parse_markdown(qry_description_file, section_number=4)))

## 3.4 Query 04

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat: de teams waarvan, gedurende een bepaald jaar, geen enkele speler meer verdiende dan `salaris`.

Een speler speelt voor een team in een bepaald jaar wanneer deze speler datzelfde jaar betaald wordt door dat team. 

Het resultaat bestaat uit teamID, naam, jaar, wins en losses in dat jaar en tenslotte het maximum salaris in dat team.

Sorteer oplopend op teamID en dan oplopend op yearID.




In [19]:
def query_04(connection, column_names, salaris="10000"):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [20]:
display(Markdown(parse_markdown(qry_description_file, section_number=5)))

## 3.5 Query 05

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat: alle teams bevat waar een (i.e., minstens 1) speler voor speelde die tussen `jaar_x` en `jaar_y` werd opgenomen in de Hall of Fame.

Een speler speelt voor een team in een bepaald jaar wanneer deze speler datzelfde jaar betaald wordt door dat team. 

Het resultaat bestaat uit teamID, naam, jaar en de rang.

Sorteer aflopend op jaar en een team slechts eenmaal voorkomt per jaar.




In [21]:
def query_05(connection, column_names, jaar_x="1960", jaar_y="1980"):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [22]:
display(Markdown(parse_markdown(qry_description_file, section_number=6)))

## 3.6 Query 06

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat alle unieke personen bevat die als manager meer dan `n_awards` hebben gewonnnen.

Van deze personen willen we de playerID, voornaam en achternaam.

Sorteer aflopend op achternaam.




In [23]:
def query_06(connection, column_names, n_awards=0):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [24]:
display(Markdown(parse_markdown(qry_description_file, section_number=7)))

## 3.7 Query 07

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat: alle personen bevat die voor een team speelden dat tussen `jaar_x` en `jaar_y` meer dan `n_wins` heeft behaald. Let op, we bedoelen hier het *totaal* aantal wins van een team in dat bepaalde interval tussen `jaar_x` en `jaar_y`, over de jaren heen dus.

Een speler speelt voor een team in een bepaald jaar wanneer deze speler datzelfde jaar betaald wordt door dat team. 

Van deze personen willen we de playerID, voornaam en achternaam en teamID.

Sorteer oplopend op playerID.




In [25]:
def query_07(connection, column_names, jaar_x=1980, jaar_y=1990, n_wins=50):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [26]:
display(Markdown(parse_markdown(qry_description_file, section_number=8)))

## 3.8 Query 08

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat voor alle teams: hun team ID, naam, *beste jaar* en het aantal homeruns en wins van dat jaar bevat.

Het *"beste jaar"* voor een team wordt hier gedefinieerd als het jaar met het hoogste aantal homeruns van dat team. In het geval van jaren met hetzelfde aantal homeruns is het meest recente jaar het beste jaar. 

Sorteer aflopend op aantal homeruns en dan aflopend op teamID.




In [27]:
def query_08(connection, column_names):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [28]:
display(Markdown(parse_markdown(qry_description_file, section_number=9)))

## 3.9 Query 09

**Beschrijving**

Het resultaat van deze functie is een Pandas dataframe dat voor een `jaar_x` alle teams bevat die voor dat jaar een bovengemiddeld aantal homeruns hadden.

Van deze teams willen we de teamID, naam, jaar, rang en natuurlijk het aantal homeruns.

Sorteer aflopend op aantal homeruns en dan aflopend op rang.




In [29]:
def query_09(connection, column_names, jaar_x="1991"):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [30]:
display(Markdown(parse_markdown(qry_description_file, section_number=10)))

## 3.10 Query 10

**Beschrijving**


Het resultaat van deze functie is een Pandas dataframe dat: de teamnaam bevat van het team (of de teams, indien meerdere) dat het meeste aantal spelers heeft die zijn opgenomen in de Hall of Fame.

Van dit team willen we de teamID, naam en het aantal spelers opgenomen in de Hall of Fame.


In [31]:
def query_10(connection, column_names):
    
    # Bouw je query
    query="""
    MAAK QUERY HIER
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [32]:
display(Markdown(parse_markdown(submissions_file, section_number=1)))

## 4. Hoe en wat in te dienen?

Nu je alle queries ingevuld (en hopelijk getest), ben je klaar om je taak in te dienen. De deadline wordt gecommuniceerd via Toledo.

1. **Maak een leeg bestand** aan, en geef het de bestandsnaam met formaat: `achternaam_voornaam_studentennummer.py`.
    - `achternaam` verwijst naar je achternaam, bijvoorbeeld `Janssens`
    - `voornaam`  verwijst naar je achternaam, bijvoorbeeld `Patrick`
    - `studentennummer` is je studentennummer zoals vermeld op je studentenkaart, bijvoorbeeld r0123456
    -  Een goede bestandsnaam is dus bijvoorbeeld: `janssens_patrick_r0123456.py`

2. Kopieer **ALLE INGEVULDE FUNCTIES EN NIETS ANDERS** naar dit bestand. Het bevat dus _enkel en alleen_ de functies:
    - query_01(connection, column_names)
    - query_02(connection, column_names, datum_x='1980-01-16')
    - etc, etc

3. **TIP**: voor de eerste 3 queries kan je je oplossing zelfs testen via de `verification.ipynb` notebook!
    - Eerst worden je functies ingelezen vanuit het zonet aangemaakte bestand
    - Die functies worden automatisch gerund, met verschillende parameters
    - De resultaten worden opgeslagen als `.csv` files (in de `out` folder)
    - Die `.csv` files worden vergeleken met de `.csv` files van de oplossing (te vinden in de `solution` folder).
    - Elke query krijgt een score toegekend. Cf. https://en.wikipedia.org/wiki/F1_score.  
    - Al dan niet sorteren is verantwoordelijk voor 10% van je score.
    - Een kort rapport wordt weergegeven die je hints kan geven over wat er mis is met je query. 
        - TP: True Positives
        - TN: True Negatives
        - FP: False Positives
    - Dit kan je helpen om je oplossing van de eerste 3 queries te debuggen, _en_ om zeker te zijn dat je `achternaam_voornaam_studentennummer.py` bestand correct werkt.
    - Voor de overige 7 queries houden we de oplossingen geheim tot na de deadline. Daar moet je dus zelf de resultaten inspecteren om jezelf ervan te overtuigen dat deze inderdaad correct zijn.
        
4. Als je oplossing definitief is, submit je je `achternaam_voornaam_studentennummer.py` bestand via Toledo. De deadline wordt gecommuniceerd via Toledo.   

5. Nogmaals, als finale submissie verwachten we dus enkel een python bestand (e.g., `achternaam_voornaam_studentennummer.py`) dat jullie ingevulde functies bevat en niks anders. Zoals zonet uitgelegd laat de `verification.ipynb` notebook je toe om zelf te checken dat je bestand aan deze standaard voldoet. **Bijgevolg: eender welke afwijking van deze standaard is absoluut onacceptabel en zal als dusdanig behandeld worden.**  
